[View in Colaboratory](https://colab.research.google.com/github/contractorwolf/tensorflow-irisdataset/blob/master/Iris_Colab.ipynb)

In [1]:
import pandas as pd
import tensorflow as tf


# verify version of tensorflow, should be 1.4+ to use new Estimators features
print('TENSORFLOW VERSION: ' + tf.__version__)

TENSORFLOW VERSION: 1.7.0


In [0]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL  = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# The remainder of this file contains a simple example of a csv parser,
#     implemented using the `Dataset` class.

# `tf.parse_csv` sets the types of the outputs to match the examples given in
#     the `record_defaults` argument.
CSV_TYPES = [[0.0], [0.0], [0.0], [0.0], [0]]

BATCH_SIZE = 50
TRAIN_STEPS = 500

#args.batch_size), steps=args.train_steps


In [0]:
def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

def _parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, record_defaults=CSV_TYPES)

    # Pack the result into a dictionary
    features = dict(zip(CSV_COLUMN_NAMES, fields))

    # Separate the label from the features
    label = features.pop('Species')

    return features, label


def csv_input_fn(csv_path, batch_size):
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(1)

    # Parse each line.
    dataset = dataset.map(_parse_line)

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [4]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [21]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
SepalLength    120 non-null float64
SepalWidth     120 non-null float64
PetalLength    120 non-null float64
PetalWidth     120 non-null float64
dtypes: float64(4)
memory usage: 3.8 KB


In [0]:
# Feature columns describe how to use the input.
my_feature_columns = []

for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [0]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

In [16]:
# Train the Model.
classifier.train(input_fn=lambda:train_input_fn(train_x, train_y, BATCH_SIZE), steps=TRAIN_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4zdhd5z_/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 501 into /tmp/tmp4zdhd5z_/model.ckpt.
INFO:tensorflow:loss = 1.6091413, step = 501
INFO:tensorflow:global_step/sec: 600.956
INFO:tensorflow:loss = 4.061643, step = 601 (0.168 sec)
INFO:tensorflow:global_step/sec: 807.061
INFO:tensorflow:loss = 3.6340642, step = 701 (0.125 sec)
INFO:tensorflow:global_step/sec: 664.339
INFO:tensorflow:loss = 1.8412668, step = 801 (0.150 sec)
INFO:tensorflow:global_step/sec: 714.226
INFO:tensorflow:loss = 3.5633256, step = 901 (0.142 sec)
INFO:tensorflow:global_step/sec: 780.134
INFO:tensorflow:loss = 1.0095496, step = 1001 (0.131 sec)
INFO:tensorflow:global_step/sec: 654.057
INFO:tensorflow:loss = 4.300429, ste

INFO:tensorflow:global_step/sec: 821.416
INFO:tensorflow:loss = 4.0569215, step = 3301 (0.118 sec)
INFO:tensorflow:global_step/sec: 769.051
INFO:tensorflow:loss = 0.7580683, step = 3401 (0.130 sec)
INFO:tensorflow:global_step/sec: 735.585
INFO:tensorflow:loss = 0.6331783, step = 3501 (0.136 sec)
INFO:tensorflow:global_step/sec: 773.084
INFO:tensorflow:loss = 3.4690738, step = 3601 (0.130 sec)
INFO:tensorflow:global_step/sec: 763.552
INFO:tensorflow:loss = 4.207726, step = 3701 (0.130 sec)
INFO:tensorflow:global_step/sec: 810.013
INFO:tensorflow:loss = 4.1233864, step = 3801 (0.128 sec)
INFO:tensorflow:global_step/sec: 792.154
INFO:tensorflow:loss = 7.430578, step = 3901 (0.123 sec)
INFO:tensorflow:global_step/sec: 742.263
INFO:tensorflow:loss = 1.2738186, step = 4001 (0.135 sec)
INFO:tensorflow:global_step/sec: 789.024
INFO:tensorflow:loss = 3.9100206, step = 4101 (0.126 sec)
INFO:tensorflow:global_step/sec: 847.534
INFO:tensorflow:loss = 3.363851, step = 4201 (0.118 sec)
INFO:tensorfl

In [17]:
# Evaluate the model.
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(test_x, test_y, BATCH_SIZE))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-17-01:44:33
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4zdhd5z_/model.ckpt-5500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-17-01:44:33
INFO:tensorflow:Saving dict for global step 5500: accuracy = 0.96666664, average_loss = 0.057669293, global_step = 5500, loss = 1.7300788

Test set accuracy: 0.967



In [19]:


# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']

predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth' : [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth' : [0.5, 1.5, 2.1],
}

predictions = classifier.predict(input_fn=lambda:eval_input_fn(predict_x, labels=None, batch_size=BATCH_SIZE))

template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4zdhd5z_/model.ckpt-5500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (100.0%), expected "Versicolor"

Prediction is "Virginica" (99.4%), expected "Virginica"
